# Rental Visualization

In [ ]:
import pandas as pd

# Forecast data
df = pd.read_csv('https://raw.githubusercontent.com/JenBanks8585/Randomdata/main/data/Realty/forecast_plot.csv', index_col = [0])
df.columns = ['zip', 'city', 'state', 'level', '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01']

df.head()

,zip,city,state,level,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01
0,1752,Marlborough,MA,low,1669,1667,1664,1661,1657
1,1752,Marlborough,MA,high,1674,1678,1682,1686,1689
2,1760,Natick,MA,low,1933,1919,1904,1889,1873
3,1760,Natick,MA,high,1942,1940,1939,1940,1940
4,1801,Woburn,MA,low,2003,1984,1966,1949,1929


In [ ]:
# Convert to long format
df_melt = pd.melt(df, id_vars=['zip', 'city', 'state', 'level'])

# Rename columns
df_melt.columns = ['zip', 'city', 'state', 'level', 'month','rent_forecast']

# Cast date to datetime object
df_melt['month']= pd.to_datetime(df_melt['month'], infer_datetime_format=True)
df_melt.head(5)

,zip,city,state,level,month,rent_forecast
0,1752,Marlborough,MA,low,2021-01-01,1669
1,1752,Marlborough,MA,high,2021-01-01,1674
2,1760,Natick,MA,low,2021-01-01,1933
3,1760,Natick,MA,high,2021-01-01,1942
4,1801,Woburn,MA,low,2021-01-01,2003


In [ ]:
# Training data

df_orig = pd.read_csv("https://raw.githubusercontent.com/JenBanks8585/Randomdata/main/data/Realty/Zip_ZORI_AllHomesPlusMultifamily_SSA%20(1).csv", parse_dates= True)
df_orig = df_orig.drop(columns =['SizeRank'], axis = 0)
print(df_orig.shape)
df_orig.head(3)

(2263, 87)


,RegionID,RegionName,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
0,61639,10025,"New York, NY",3025.0,3039.0,3054.0,3068.0,3083.0,3097.0,3112.0,3126.0,3140.0,3155.0,3169.0,3182.0,3196.0,3207.0,3217.0,3227.0,3233.0,3239.0,3244.0,3248.0,3252.0,3257.0,3260.0,3264.0,3268.0,3270.0,3272.0,3274.0,3276.0,3277.0,3279.0,3277.0,3276.0,3275.0,3275.0,3274.0,3274.0,...,3267.0,3264.0,3263.0,3261.0,3260.0,3260.0,3260.0,3260.0,3262.0,3264.0,3267.0,3270.0,3274.0,3277.0,3281.0,3286.0,3290.0,3298.0,3305.0,3313.0,3324.0,3334.0,3345.0,3353.0,3361.0,3369.0,3361.0,3353.0,3346.0,3311.0,3277.0,3242.0,3201.0,3159.0,3117.0,3073.0,3029.0,2985.0,2938.0,2891.0
1,84654,60657,"Chicago, IL",1580.0,1585.0,1591.0,1596.0,1602.0,1607.0,1612.0,1618.0,1623.0,1628.0,1633.0,1638.0,1643.0,1648.0,1653.0,1657.0,1662.0,1666.0,1670.0,1675.0,1679.0,1684.0,1689.0,1694.0,1699.0,1703.0,1708.0,1713.0,1717.0,1722.0,1726.0,1730.0,1735.0,1739.0,1742.0,1745.0,1748.0,...,1761.0,1762.0,1763.0,1765.0,1766.0,1766.0,1766.0,1767.0,1767.0,1767.0,1767.0,1768.0,1769.0,1770.0,1773.0,1776.0,1778.0,1783.0,1788.0,1793.0,1799.0,1806.0,1812.0,1817.0,1822.0,1827.0,1828.0,1829.0,1830.0,1825.0,1820.0,1816.0,1810.0,1803.0,1797.0,1791.0,1784.0,1777.0,1770.0,1762.0
2,61637,10023,"New York, NY",3156.0,3164.0,3172.0,3180.0,3188.0,3195.0,3203.0,3210.0,3218.0,3225.0,3231.0,3237.0,3242.0,3248.0,3254.0,3260.0,3267.0,3274.0,3281.0,3287.0,3294.0,3300.0,3305.0,3309.0,3314.0,3314.0,3313.0,3313.0,3309.0,3305.0,3301.0,3298.0,3295.0,3292.0,3293.0,3295.0,3296.0,...,3304.0,3303.0,3300.0,3298.0,3296.0,3295.0,3295.0,3295.0,3299.0,3303.0,3306.0,3310.0,3314.0,3317.0,3323.0,3328.0,3333.0,3341.0,3350.0,3358.0,3366.0,3375.0,3384.0,3391.0,3397.0,3404.0,3397.0,3390.0,3383.0,3349.0,3316.0,3282.0,3241.0,3201.0,3160.0,3116.0,3073.0,3029.0,2983.0,2937.0


In [ ]:
# labels
df_1= df_orig.iloc[:,1:3]

# Grabbing data from 2018 onwards
df_2= df_orig.iloc[:,60:]

# Concatenate
df_me = pd.concat([df_1, df_2], axis = 1)

# Transform to long format
df_mew = pd.melt(df_me, id_vars=['RegionName', 'MsaName'])

# Grab columns
df_mew.columns =['zip', 'cityname', 'date', 'rent_forecast']

# Cast date to datetime object
df_mew['date']= pd.to_datetime(df_mew['date'])

df_mew.head()

,zip,cityname,date,rent_forecast
0,10025,"New York, NY",2018-10-01,3277.0
1,60657,"Chicago, IL",2018-10-01,1770.0
2,10023,"New York, NY",2018-10-01,3317.0
3,77494,"Houston, TX",2018-10-01,1753.0
4,60614,"Chicago, IL",2018-10-01,1928.0


In [ ]:
import plotly.graph_objects as go

def plotme(zip):
  zips = list(df_me['RegionName'])

  if zip in zips:
    title = df_me.loc[df_me['RegionName']==zip, 'MsaName'].item()
    marl_low = df_melt[(df_melt['zip']==zip) & (df_melt['level']=='low')]
    marl_high = df_melt[(df_melt['zip']==zip) & (df_melt['level']=='high')]
    orig_vals = df_mew.loc[df_mew['zip']== zip, ['date','rent_forecast']]  

    fig = go.Figure()

    fig.add_trace(go.Scatter(name="Observed", 
                            x=list(orig_vals['date']), 
                            y=list(orig_vals['rent_forecast']),
        fill=None,
        mode='markers',
        line_color='red'))
    
    fig.add_trace(go.Scatter(name="Low Forecast", 
                            x=list(marl_low['month']), 
                            y=list(marl_low['rent_forecast']),
        fill=None,
        mode='lines',
        line_color='green'))

    fig.add_trace(go.Scatter(name="High Forecast",
                            x=list(marl_high['month']), 
                            y=list(marl_high['rent_forecast']),
        fill='tonexty', 
        mode='lines', line_color='green'))  
    
    fig.update_layout({
        'autosize':False,
        'plot_bgcolor':'rgba(2,0,0,5)',
        'paper_bgcolor':'rgba(2,0,0,5)',
        'font_color':"white",
        'title': f'Region name: {title}',
        'yaxis_title': 'Rent ($)',
        'xaxis_title': 'Date'
        })
    
    fig.update_yaxes(automargin = True, gridcolor = "gray")
    fig.update_xaxes(automargin = True, gridcolor = "gray")
   
    #fig.show()
    return fig.to_json()

  else:
    return {"message":"No Date for this location"}


In [ ]:
plotme(10010)

'{"data":[{"line":{"color":"red"},"mode":"markers","name":"Observed","type":"scatter","x":["2018-10-01T00:00:00","2018-11-01T00:00:00","2018-12-01T00:00:00","2019-01-01T00:00:00","2019-02-01T00:00:00","2019-03-01T00:00:00","2019-04-01T00:00:00","2019-05-01T00:00:00","2019-06-01T00:00:00","2019-07-01T00:00:00","2019-08-01T00:00:00","2019-09-01T00:00:00","2019-10-01T00:00:00","2019-11-01T00:00:00","2019-12-01T00:00:00","2020-01-01T00:00:00","2020-02-01T00:00:00","2020-03-01T00:00:00","2020-04-01T00:00:00","2020-05-01T00:00:00","2020-06-01T00:00:00","2020-07-01T00:00:00","2020-08-01T00:00:00","2020-09-01T00:00:00","2020-10-01T00:00:00","2020-11-01T00:00:00","2020-12-01T00:00:00"],"y":[3872.0,3880.0,3888.0,3896.0,3905.0,3913.0,3922.0,3930.0,3938.0,3946.0,3952.0,3957.0,3962.0,3950.0,3938.0,3926.0,3881.0,3837.0,3792.0,3739.0,3686.0,3633.0,3577.0,3521.0,3465.0,3406.0,3347.0]},{"line":{"color":"green"},"mode":"lines","name":"Low Forecast","type":"scatter","x":["2021-01-01T00:00:00","2021-02-01

In [ ]:
def plotme_viz(zip):
  zips = list(df_me['RegionName'])

  if zip in zips:
    title = df_me.loc[df_me['RegionName']==zip, 'MsaName'].item()
    marl_low = df_melt[(df_melt['zip']==zip) & (df_melt['level']=='low')]
    marl_high = df_melt[(df_melt['zip']==zip) & (df_melt['level']=='high')]
    orig_vals = df_mew.loc[df_mew['zip']== zip, ['date','rent_forecast']]  

    fig = go.Figure()

    fig.add_trace(go.Scatter(name="Observed", 
                            x=list(orig_vals['date']), 
                            y=list(orig_vals['rent_forecast']),
        fill=None,
        mode='markers',
        line_color='red'))
    
    fig.add_trace(go.Scatter(name="Low Forecast", 
                            x=list(marl_low['month']), 
                            y=list(marl_low['rent_forecast']),
        fill=None,
        mode='lines',
        line_color='green'))

    fig.add_trace(go.Scatter(name="High Forecast",
                            x=list(marl_high['month']), 
                            y=list(marl_high['rent_forecast']),
        fill='tonexty', 
        mode='lines', line_color='green'))  
    
    fig.update_layout({
        'autosize':False,
        'plot_bgcolor':'rgba(2,0,0,5)',
        'paper_bgcolor':'rgba(2,0,0,5)',
        'font_color':"white",
        'title': f'Region name: {title}',
        'yaxis_title': 'Rent ($)',
        'xaxis_title': 'Date'
        })
    
    fig.update_yaxes(automargin = True, gridcolor = "gray")
    fig.update_xaxes(automargin = True, gridcolor = "gray")
   
    fig.show()
    

  else:
    return {"message":"No Date for this location"}

In [ ]:
plotme_viz(1752)

In [ ]:
plotme_viz(10023)

# Pollution Visualization

In [ ]:
import pandas as pd

air = pd.read_csv('https://raw.githubusercontent.com/JenBanks8585/Randomdata/main/data/Pollution/daily_aqi_by_county_2020.csv')
air = air[["State Name", "county Name", "Date", "AQI", "Category", "Defining Parameter"]]
air.tail()

,State Name,county Name,Date,AQI,Category,Defining Parameter
217709,Wyoming,Weston,2020-06-26,43,Good,Ozone
217710,Wyoming,Weston,2020-06-27,61,Moderate,Ozone
217711,Wyoming,Weston,2020-06-28,49,Good,Ozone
217712,Wyoming,Weston,2020-06-29,54,Moderate,Ozone
217713,Wyoming,Weston,2020-06-30,44,Good,Ozone


In [ ]:
# Transform to long format
air_melt = pd.melt(air, id_vars=["State Name", "county Name", "Category", "Date", "Defining Parameter"])
air_melt = air_melt.drop(columns = "variable", axis =0)
air_melt['Date'] = pd.to_datetime(air_melt['Date'])
air_melt['Month']=  pd.DatetimeIndex(air_melt['Date']).month
air_melt.tail(2)

,State Name,county Name,Category,Date,Defining Parameter,value,Month
217712,Wyoming,Weston,Moderate,2020-06-29,Ozone,54,6
217713,Wyoming,Weston,Good,2020-06-30,Ozone,44,6


In [ ]:
#air_melt_grouped = air_melt.groupby(["county Name","State Name", "Defining Parameter"]).mean()
air_melt_grouped = air_melt.groupby(["Date","county Name","State Name","Month", "Defining Parameter", "Category"]).mean()
air_melt_grouped['value']= air_melt_grouped['value'].astype(int)
air_melt_grouped = air_melt_grouped.reset_index()
air_melt_grouped.head()

,Date,county Name,State Name,Month,Defining Parameter,Category,value
0,2020-01-01,Ada,Idaho,1,Ozone,Good,28
1,2020-01-01,Adair,Oklahoma,1,Ozone,Good,31
2,2020-01-01,Adams,Colorado,1,NO2,Good,45
3,2020-01-01,Adams,Ohio,1,PM2.5,Good,21
4,2020-01-01,Adams,Pennsylvania,1,Ozone,Good,33


In [ ]:
aqi = air_melt_grouped[(air_melt_grouped['county Name']=="Ada") & (air_melt_grouped['State Name']=="Idaho")]
aqi.head()

,Date,county Name,State Name,Month,Defining Parameter,Category,value
0,2020-01-01,Ada,Idaho,1,Ozone,Good,28
913,2020-01-02,Ada,Idaho,1,NO2,Good,30
1744,2020-01-03,Ada,Idaho,1,NO2,Good,35
2575,2020-01-04,Ada,Idaho,1,NO2,Good,33
3491,2020-01-05,Ada,Idaho,1,PM2.5,Good,30


In [ ]:
set(air_melt_grouped["Category"])

{'Good',
 'Hazardous',
 'Moderate',
 'Unhealthy',
 'Unhealthy for Sensitive Groups',
 'Very Unhealthy'}

In [ ]:
import plotly.express as px

def plot_pollution2(county, state, param):
  aqi = air_melt_grouped[(air_melt_grouped['county Name']==county) & (air_melt_grouped['State Name']==state) & (air_melt_grouped['Defining Parameter']==param)]
  fig = px.scatter(aqi, x ="Date", y = "value", color = "Defining Parameter" )

  fig.show()

plot_pollution2("Ada", "Idaho", "Ozone")


In [ ]:
import plotly.graph_objects as go

def plot_pollution(county, state):
  #counties = set(air_melt['county Name'])


    #title = air_melt.loc[air_melt['county Name']==county, 'MsaName'].item()
  aqi = air_melt_grouped[(air_melt_grouped['county Name']==county) & (air_melt_grouped['State Name']==state)]

  fig = go.Figure()

  fig.add_trace(go.Scatter(name="Observed", 
                             x=list(aqi['Month']), 
                             y=list(aqi['value']),                          
                                    
        fill=None,
        mode='markers',
        line_color='red'))
    
  fig.update_layout({
        'autosize':False,
        'plot_bgcolor':'rgba(2,0,0,5)',
        'paper_bgcolor':'rgba(2,0,0,5)',
        'font_color':"white",
        'title': f'Region name: {county}, {state}',
        'yaxis_title': 'Air Quality Index',
        'xaxis_title': 'Month in 2020'
        })
    
  fig.update_yaxes(automargin = True, gridcolor = "gray")
  fig.update_xaxes(automargin = True, gridcolor = "gray")
   
  fig.show()

  #else:
  #  return {"message":"No Date for this location"}

In [ ]:
plot_pollution('Ada', 'Idaho')

In [ ]:
aqicbs = pd.read_csv("daily_aqi_by_cbsa_2020.csv")
aqicbs.loc[1034:1170, :]

In [ ]:
import os

import pandas as pd 
from fastapi import APIRouter, Depends
from pydantic import BaseModel, SecretStr
import sqlalchemy
import json

from dotenv import load_dotenv


load_dotenv()
database_url = os.getenv('PRODUCTION_DATABASE_URL')
engine = sqlalchemy.create_engine(database_url)
query = '''SELECT RTRIM(CITIES.city_name) as city_name, 
                  RENTAL_FORECAST.rental_forecast,
                  RTRIM(STATES.state_name) as state_name    
            FROM CITIES
            INNER JOIN RENTAL_FORECAST 
                ON CITIES.city_id=RENTAL_FORECAST.city_id  
            INNER JOIN STATES
                ON CITIES.state_id = STATES.state_id
            '''

df =pd.read_sql(query, database_url)
print(df.head())

# Population Visualization

In [ ]:
population =pd.read_csv("https://raw.githubusercontent.com/JenBanks8585/Randomdata/main/data/Population%20data/population_for_visual.csv", index_col = [0])
population['city_name']=population['city_name'].apply(lambda x: x.strip())
population['state_abbreviation']=population['state_abbreviation'].apply(lambda x: x.strip())
population.head()

,city_name,population,year,state_abbreviation
0,Abbeville,2699,2010,AL
1,Adamsville,4500,2010,AL
2,Addison,751,2010,AL
3,Akron,355,2010,AL
4,Alabaster,31209,2010,AL


In [ ]:
import plotly.graph_objects as go

def plotpop(city1, state1, city2= None, state2= None):
  cities = list(population['city_name'])

  if city1 in cities:
    df_subset = population[(population['city_name']==city1) & (population['state_abbreviation']==state1)]
    df_subset2 = population[(population['city_name']==city2) & (population['state_abbreviation']==state2)]

    fig = go.Figure()

    fig.add_trace(go.Scatter(name=f"{city1}, {state1}", 
                            x=list(df_subset['year']), 
                            y=list(df_subset['population']),
        fill=None,
        mode='lines',
        line_color='red'))
    
    fig.add_trace(go.Scatter(name=f"{city2}, {state2}", 
                            x=list(df_subset2['year']), 
                            y=list(df_subset2['population']),
        fill=None,
        mode='lines',
        line_color='blue'))
    

    fig.update_layout({
        'autosize':False,
        'plot_bgcolor':'rgba(2,0,0,5)',
        'paper_bgcolor':'rgba(2,0,0,5)',
        'font_color':"white",
        'title': f'Population {city1}, {state1} and {city2}, {state2}',
        'yaxis_title': 'Population',
        'xaxis_title': 'Year'
        })
    
    fig.update_yaxes(automargin = True, gridcolor = "gray", rangemode="tozero")
    fig.update_xaxes(automargin = True, gridcolor = "gray")
   
    fig.show()
    return fig.to_json()

  else:
    return {"message":"No Date for this location"}


In [ ]:
plotpop('San Diego', 'CA', 'San Francisco', "CA")

'{"data":[{"line":{"color":"red"},"mode":"lines","name":"San Diego, CA","type":"scatter","x":[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],"y":[1305906,1319592,1336776,1355320,1375831,1387323,1402089,1412621,1421917,1423851]},{"line":{"color":"blue"},"mode":"lines","name":"San Francisco, CA","type":"scatter","x":[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],"y":[805505,815650,828876,839572,850750,863010,871512,878040,880696,881549]}],"layout":{"autosize":false,"font":{"color":"white"},"paper_bgcolor":"rgba(2,0,0,5)","plot_bgcolor":"rgba(2,0,0,5)","template":{"data":{"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"bar"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"barpolar"}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","lineco

In [ ]:
air_melt_grouped = air_melt.groupby(["Date","county Name","State Name","Month", "Defining Parameter", "Category"]).mean()
air_melt_grouped['value']= air_melt_grouped['value'].astype(int)
air_melt_grouped = air_melt_grouped.reset_index()

In [ ]:
pop = population.copy()
pop = pop.drop(columns =['city_name'], axis = 0)
pop_grouped = pop.groupby(['state_abbreviation', 'year']).sum()
pop_grouped=pop_grouped.reset_index()

pop_grouped.tail()

,state_abbreviation,year,population
495,WY,2015,404908
496,WY,2016,403325
497,WY,2017,398845
498,WY,2018,397338
499,WY,2019,397716


In [ ]:
pop_year = pop_grouped[pop_grouped['year']==2019]
pop_year.head()

,state_abbreviation,year,population
9,AK,2019,166865
19,AL,2019,2998987
29,AR,2019,2001152
39,AZ,2019,5791407
49,CA,2019,32965607


In [ ]:
import plotly.graph_objects as go

import pandas as pd

def plot_population_by_year(year):

  pop_year = pop_grouped[pop_grouped['year']==year]

  fig = go.Figure(data=go.Choropleth(
      locations=pop_year['state_abbreviation'], 
      z = pop_year['population'].astype(float), 
      locationmode = 'USA-states', 
      colorscale = 'Blues',

  ))

  fig.update_layout({
      'title':f'{year} US Population by State',
      'geo_scope':'usa', 
      'autosize':False,
      'plot_bgcolor':'rgba(2,0,0,5)',
      'paper_bgcolor':'rgba(2,0,0,5)',
  })

  fig.show()

plot_population_by_year(2011)

